In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
start_url = "https://trak.in/india-startup-funding-investment-2015/"
html_response = requests.get(start_url)
html_response.status_code # code 200 means connecition was sucessful and it is now active

200

In [ ]:
soup = BeautifulSoup(html_response.content, 'html.parser')
#print(soup.prettify())

In [ ]:
more_data_urls = [start_url]

for h3_tag in soup.find_all(name="h3"):
    more_data_urls.append(h3_tag.find(name='a').get('href'))

more_data_urls[:5]

['https://trak.in/india-startup-funding-investment-2015/',
 'https://trak.in/indian-startup-funding-and-investment-chart-january-2018/',
 'https://trak.in/indian-startup-funding-and-investment-chart-february-2018/',
 'https://trak.in/indian-startup-funding-and-investment-chart-march-2018/',
 'https://trak.in/indian-startup-funding-investment-chart-april-2018/']

In [ ]:
import pandas as pd

new_row_list = []
column_name = ['Sr. No.', 'Date (dd/mm/yyyy)', 'Startup Name', 'Industry/ Vertical', 'Sub-Vertical', 'City / Location', 'Investors’ Name', 'Investment Type', 'Amount (in USD)']
more_data_urls = set(more_data_urls)

urls_count = 1
for url in more_data_urls:
    html_response = requests.get(url)
    html_response.status_code
    soup = BeautifulSoup(html_response.content, 'html.parser')
    
    class_list = []
    for element in soup.find_all(class_=True):
        class_list.extend(element["class"])
    class_list = [cls for cls in class_list if 'tablepress-id-' in cls] 
    
    if len(class_list) < 1:
        skip_first_row = True
        class_list.append(None)
        for class_ in class_list:
            tbl=soup.find(name='table') #, class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                if skip_first_row == True:
                    skip_first_row = False
                    continue
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-old:", class_, len(new_row_list), n_rows, url)
    else:
        for class_ in class_list:
            tbl=soup.find(name='table', class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-new :", class_, len(new_row_list), n_rows, url)

data = pd.DataFrame(new_row_list, columns=column_name)
print("Data shape :", data.shape)

Data shape : (3511, 9)


In [ ]:
data  

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry/ Vertical,Sub-Vertical,City / Location,Investors’ Name,Investment Type,Amount (in USD)
0,1,02/02/2017,Clodura,Technology,Intelligent Sales Assistant,Pune,Undisclosed Investors,Seed Funding,N/A
1,2,03/02/2017,Stockroom.io,Consumer Internet,Curated Platform for Developers,Hyderabad,"Satya D Sinha, Gyanendra Singh",Seed Funding,N/A
2,3,06/02/2017,Capital Float,Consumer Internet,Online lending platform,Bangalore,IFMR Capital Finance,Private Equity,"2,500,000"
3,4,06/02/2017,iManageMyHotel,Technology,Hotel Management Solution,Kolkata,Jaarvis Accelerator,Seed Funding,N/A
4,5,07/02/2017,Inner Hour,Consumer Internet,Online Counselling Platform,Mumbai,"Batlivala & Karani Securities, Venture Works",Seed Funding,"450,000"
...,...,...,...,...,...,...,...,...,...
3506,97,30/06/2016,Campus Diaries,Consumer Internet,Student focussed Content Discovery portal,Bangalore,Aarin Capital,Seed Funding,"220,000"
3507,98,30/06/2016,Scripbox,Consumer Internet,Online Mutual Fund Investment platform,Bangalore,"Omidyar Network, Accel Partners & existing inv...",Private Equity,N/A
3508,99,30/06/2016,Biryani Blues,Food & Beverage,QSR Chain,Gurgaon,Carpediem Capital & Others,Private Equity,"368,000"
3509,100,30/06/2016,,,,,,,


In [ ]:
data_copy = data.copy()
data.columns

Index(['Sr. No.', 'Date (dd/mm/yyyy)', 'Startup Name', 'Industry/ Vertical',
       'Sub-Vertical', 'City / Location', 'Investors’ Name', 'Investment Type',
       'Amount (in USD)'],
      dtype='object')

In [ ]:
data = data_copy
data = data.dropna(how='all')
data.shape

(3511, 9)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3511 entries, 0 to 3510
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sr. No.             3511 non-null   object
 1   Date (dd/mm/yyyy)   3511 non-null   object
 2   Startup Name        3511 non-null   object
 3   Industry/ Vertical  3511 non-null   object
 4   Sub-Vertical        3511 non-null   object
 5   City / Location     3511 non-null   object
 6   Investors’ Name     3511 non-null   object
 7   Investment Type     3337 non-null   object
 8   Amount (in USD)     3334 non-null   object
dtypes: object(9)
memory usage: 247.0+ KB
